In [3]:
# 1. 필요한 라이브러리 설치 (statsmodels는 보통 코랩에 기본 설치되어 있습니다)
!pip install pandas gspread google-auth-oauthlib statsmodels -q

# 2. 라이브러리 임포트 및 구글 인증
import pandas as pd
import numpy as np
import gspread
from google.colab import auth
from google.auth import default
from statsmodels.stats.inter_rater import fleiss_kappa

print("Google 계정 인증을 시작합니다...")
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ 인증 성공!")
except Exception as e:
    print(f"❌ 인증에 실패했습니다: {e}")
    exit()

# 3. '케파 통계' 구글 스프레드시트 데이터 불러오기
try:
    spreadsheet_name = '케파통계'
    print(f"\n'{spreadsheet_name}' 스프레드시트를 불러옵니다...")

    worksheet = gc.open(spreadsheet_name).sheet1
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)

    print("✅ 스프레드시트 로드 완료!")
    print(f" > 총 {len(df)}개의 라벨링 데이터를 확인했습니다.")

except Exception as e:
    print(f"❌ 스프레드시트를 불러오는 중 오류가 발생했습니다: {e}")
    exit()


# 4. Fleiss's Kappa 계산을 위한 데이터 변환 함수
def prepare_for_fleiss(dataframe, rater_cols, category_list):
    """
    데이터프레임을 Fleiss's Kappa 계산에 필요한 집계 테이블 형태로 변환합니다.
    (각 항목(row)에 대해, 각 카테고리(col)를 선택한 평가자의 수를 집계)
    """
    # 카테고리별로 정수 인덱스를 매핑
    cat_to_int = {category: i for i, category in enumerate(category_list)}

    # (항목 수, 카테고리 수) 모양의 0으로 채워진 배열 생성
    table = np.zeros((len(dataframe), len(category_list)), dtype=int)

    # 데이터프레임의 각 행(각 기사)을 순회
    for i, row in dataframe[rater_cols].iterrows():
        # 각 평가자의 라벨을 순회
        for label in row:
            if label in cat_to_int:
                # 해당하는 카테고리의 카운트를 1 증가
                table[i, cat_to_int[label]] += 1
    return table


# 5. Fleiss's Kappa 계산 및 결과 출력
print("\n" + "="*45)
print("      평가자 간 신뢰도(Fleiss's Kappa) 분석 결과")
print("="*45)

# --- 5-1. 정당 편향 신뢰도 계산 ---
party_rater_columns = ['박세온 정당', '박상춘 정당', '최수진 정당']
party_categories = ['민주당', '국민의힘', '그외']

if all(col in df.columns for col in party_rater_columns):
    print("\n📊 [정당 편향]에 대한 신뢰도를 계산합니다...")

    # 1. 데이터 변환
    party_table = prepare_for_fleiss(df, party_rater_columns, party_categories)

    # 2. Fleiss's Kappa 계산
    kappa_party = fleiss_kappa(party_table)

    print("\n--- 정당 편향 Kappa 통계 결과 ---")
    print(f"Fleiss's Kappa: {kappa_party:.4f}")
else:
    print(f"\n❌ 경고: 정당 편향 분석에 필요한 열 {party_rater_columns} 중 일부를 찾을 수 없습니다.")


# --- 5-2. 감성 편향 신뢰도 계산 ---
sentiment_rater_columns = ['박세온 감정', '박상춘 감정', '최수진 감정']
sentiment_categories = ['긍정', '부정', '중립']

if all(col in df.columns for col in sentiment_rater_columns):
    print("\n\n📊 [감성 편향]에 대한 신뢰도를 계산합니다...")

    # 1. 데이터 변환
    sentiment_table = prepare_for_fleiss(df, sentiment_rater_columns, sentiment_categories)

    # 2. Fleiss's Kappa 계산
    kappa_sentiment = fleiss_kappa(sentiment_table)

    print("\n--- 감성 편향 Kappa 통계 결과 ---")
    print(f"Fleiss's Kappa: {kappa_sentiment:.4f}")
else:
    print(f"\n❌ 경고: 감성 편향 분석에 필요한 열 {sentiment_rater_columns} 중 일부를 찾을 수 없습니다.")

Google 계정 인증을 시작합니다...
✅ 인증 성공!

'케파통계' 스프레드시트를 불러옵니다...
✅ 스프레드시트 로드 완료!
 > 총 500개의 라벨링 데이터를 확인했습니다.

      평가자 간 신뢰도(Fleiss's Kappa) 분석 결과

📊 [정당 편향]에 대한 신뢰도를 계산합니다...

--- 정당 편향 Kappa 통계 결과 ---
Fleiss's Kappa: 0.8285


📊 [감성 편향]에 대한 신뢰도를 계산합니다...

--- 감성 편향 Kappa 통계 결과 ---
Fleiss's Kappa: 0.7974


In [2]:
# 1. 라이브러리 임포트 및 구글 인증 (이전과 동일)
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

print("Google 계정 인증을 시작합니다...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ 인증 성공!")

# 2. 데이터 불러오기 (이전과 동일)
try:
    spreadsheet_name = '케파통계'
    print(f"\n'{spreadsheet_name}' 스프레드시트를 불러옵니다...")
    worksheet = gc.open(spreadsheet_name).sheet1
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    print("✅ 스프레드시트 로드 완료!")
except Exception as e:
    print(f"❌ 스프레드시트 로드에 실패했습니다: {e}")
    exit()

# 3. 데이터 검사
print("\n" + "="*40)
print("      데이터 무결성 검사 시작")
print("="*40)

# 검사할 열과 카테고리 정의
party_rater_columns = ['박세온 정당', '박상춘 정당', '최수진 정당']
party_categories = ['민주당', '국민의힘', '그외']

problematic_rows = []

# 각 행을 순회하며 문제 확인
for index, row in df.iterrows():
    # 3명의 평가 데이터 가져오기
    ratings = [row.get(col, '') for col in party_rater_columns]

    # 빈 칸이 있거나, 정의된 카테고리 외의 값이 있는지 확인
    for rating in ratings:
        if rating not in party_categories:
            problematic_rows.append({
                'row_index': index + 2,  # 스프레드시트의 실제 행 번호 (헤더 포함)
                'ratings': {col: row.get(col, '!!!빈 값!!!') for col in party_rater_columns}
            })
            break # 문제가 발견되면 다음 행으로

if problematic_rows:
    print(f"\n❌ 총 {len(problematic_rows)}개의 행에서 빈 칸 또는 오타를 발견했습니다.")
    print("아래 목록을 확인하고 '케파 통계' 스프레드시트에서 직접 수정해주세요.\n")
    for problem in problematic_rows:
        print(f"  - 스프레드시트 행 번호: {problem['row_index']}, 데이터: {problem['ratings']}")
else:
    print("\n✅ 데이터에 빈 칸이나 오타가 없습니다. 이전 Kappa 계산 코드를 다시 실행해보세요.")

Google 계정 인증을 시작합니다...
✅ 인증 성공!

'케파통계' 스프레드시트를 불러옵니다...
✅ 스프레드시트 로드 완료!

      데이터 무결성 검사 시작

✅ 데이터에 빈 칸이나 오타가 없습니다. 이전 Kappa 계산 코드를 다시 실행해보세요.


In [1]:
# 1. 라이브러리 임포트 및 구글 인증 (이전과 동일)
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default

print("Google 계정 인증을 시작합니다...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ 인증 성공!")

# 2. 데이터 불러오기 (이전과 동일)
try:
    spreadsheet_name = '케파통계'
    print(f"\n'{spreadsheet_name}' 스프레드시트를 불러옵니다...")
    worksheet = gc.open(spreadsheet_name).sheet1
    data = worksheet.get_all_records()
    df = pd.DataFrame(data)
    print("✅ 스프레드시트 로드 완료!")
except Exception as e:
    print(f"❌ 스프레드시트 로드에 실패했습니다: {e}")
    exit()

# 3. '감성' 데이터 무결성 검사
print("\n" + "="*40)
print("      감성 데이터 무결성 검사 시작")
print("="*40)

# 검사할 열과 카테고리 정의
sentiment_rater_columns = ['박세온 감정', '박상춘 감정', '최수진 감정']
sentiment_categories = ['긍정', '부정', '중립']

problematic_rows_sentiment = []

# '감성' 관련 열들이 모두 존재하는지 확인
if not all(col in df.columns for col in sentiment_rater_columns):
    print(f"❌ 오류: {sentiment_rater_columns} 중 하나 이상의 열을 파일에서 찾을 수 없습니다.")
    exit()


# 각 행을 순회하며 문제 확인
for index, row in df.iterrows():
    ratings = [row.get(col, '') for col in sentiment_rater_columns]

    # 빈 칸이 있거나, 정의된 카테고리 외의 값이 있는지 확인
    for rating in ratings:
        if rating not in sentiment_categories:
            problematic_rows_sentiment.append({
                'row_index': index + 2,  # 스프레드시트의 실제 행 번호 (헤더 포함)
                'ratings': {col: row.get(col, '!!!빈 값!!!') for col in sentiment_rater_columns}
            })
            break # 문제가 발견되면 다음 행으로

if problematic_rows_sentiment:
    print(f"\n❌ [감성] 데이터에서 총 {len(problematic_rows_sentiment)}개의 문제를 발견했습니다.")
    print("아래 목록을 확인하고 '케파 통계' 스프레드시트에서 직접 수정해주세요.\n")
    for problem in problematic_rows_sentiment:
        print(f"  - 스프레드시트 행 번호: {problem['row_index']}, 데이터: {problem['ratings']}")
else:
    print("\n✅ [감성] 데이터에 빈 칸이나 오타가 없습니다. 이전 Kappa 계산 코드를 다시 실행해보세요.")

Google 계정 인증을 시작합니다...
✅ 인증 성공!

'케파통계' 스프레드시트를 불러옵니다...
✅ 스프레드시트 로드 완료!

      감성 데이터 무결성 검사 시작

❌ [감성] 데이터에서 총 2개의 문제를 발견했습니다.
아래 목록을 확인하고 '케파 통계' 스프레드시트에서 직접 수정해주세요.

  - 스프레드시트 행 번호: 31, 데이터: {'박세온 감정': '긍정', '박상춘 감정': '긍정', '최수진 감정': '그외'}
  - 스프레드시트 행 번호: 387, 데이터: {'박세온 감정': '부정', '박상춘 감정': '중립립', '최수진 감정': '부정'}
